# JSON Agent

Agent for interacting with JSON blob

In [1]:
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"
import yaml

from langchain.agents import SQLDatabaseAgent, JsonAgent
from langchain.tools.sql_database.toolkit import SQLDatabaseToolkit
from langchain.tools.json.tool import JsonSpec
from langchain.tools.json.toolkit import JsonSpecToolkit
from langchain.tools.requests.toolkit import RequestsToolkit
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.requests import RequestsWrapper

In [2]:
with open("../../../../notebooks/openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)

request_toolkit = RequestsToolkit(requests_wrapper=RequestsWrapper())
json_toolkit = JsonSpecToolkit(spec=JsonSpec(dict_=data, max_value_length=4000))

In [3]:
agent = JsonAgent.from_llm_and_tools(
    llm=OpenAI(temperature=0),
    tools=json_toolkit.get_tools(),
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=json_toolkit.get_tools(), verbose=True
)

In [5]:
agent_executor.run("What are the required parameters in the request body to the /completions endpoint?")



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the paths key to see what endpoints exist
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the /completions endpoint to see what parameters are required
Action: json_spec_list_keys
Action Input: data["paths"]["/completions"]
Observation: ['post']
Thought: I should look at the post key to see what parameters are required
Actio

 I now know the final answer
Final Answer: The required parameters in the request body to the /completions endpoint are 'model'.

> Finished chain.


"The required parameters in the request body to the /completions endpoint are 'model'."

In [4]:
agent_executor.run("Write me an example request to the completions endpoint using curl. The prompt should be 'tell me a joke'.")



> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the paths key to see what endpoints I have access to
Action: json_spec_list_keys
Action Input: data["paths"]
Observation: ['/engines', '/engines/{engine_id}', '/completions', '/edits', '/images/generations', '/images/edits', '/images/variations', '/embeddings', '/engines/{engine_id}/search', '/files', '/files/{file_id}', '/files/{file_id}/content', '/answers', '/classifications', '/fine-tunes', '/fine-tunes/{fine_tune_id}', '/fine-tunes/{fine_tune_id}/cancel', '/fine-tunes/{fine_tune_id}/events', '/models', '/models/{model}', '/moderations']
Thought: I should look at the /completions endpoint to see what parameters I need to pass
Action: json_spec_list_keys
Action Input: data["paths"]["/completions"]
Observation: ['post']
Thought: I should look at the post key to see what parameters I nee

 I now know the parameters I need to pass
Final Answer: curl -X POST "https://api.openai.com/v1/engines/davinci/completions" -H "Content-Type: application/json" -d '{"model": "davinci:2020-02-10", "prompt": "tell me a joke", "max_tokens": 50, "temperature": 0.7, "top_p": 0.9, "n": 3, "stream": false, "logprobs": 0, "echo": false, "stop": "\n", "presence_penalty": 0.0, "frequency_penalty": 0.0, "best_of": 1, "logit_bias": 0.0, "user": null}'

> Finished chain.


'curl -X POST "https://api.openai.com/v1/engines/davinci/completions" -H "Content-Type: application/json" -d \'{"model": "davinci:2020-02-10", "prompt": "tell me a joke", "max_tokens": 50, "temperature": 0.7, "top_p": 0.9, "n": 3, "stream": false, "logprobs": 0, "echo": false, "stop": "\\n", "presence_penalty": 0.0, "frequency_penalty": 0.0, "best_of": 1, "logit_bias": 0.0, "user": null}\''